<a href="https://colab.research.google.com/github/ratusilma/Sentiment-Analysis-using-Transformer-Model/blob/main/Tugas_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW # Corrected import for AdamW
from tqdm import tqdm
import os

# ==========================================
# 1. PEMBUATAN DATASET (SIMULASI)
# ==========================================

def generate_mock_data():
    # Contoh template kalimat untuk data latih
    pos_templates = ["Saya sangat senang hari ini", "Produk ini sangat bagus", "Layanan yang memuaskan", "Sangat merekomendasikan ini", "Luar biasa sekali hasilnya"]
    neg_templates = ["Saya sangat kecewa", "Produk ini buruk sekali", "Layanan yang mengecewakan", "Jangan beli barang ini", "Sangat buruk kualitasnya"]
    neu_templates = ["Saya sedang di rumah", "Hari ini cuaca biasa saja", "Buku itu ada di meja", "Saya sedang membaca berita", "Ponsel ini berwarna hitam"]

    # Membuat 100 Positif, 100 Negatif, 100 Netral (Total 300)
    train_data = []
    for _ in range(20):
        for text in pos_templates: train_data.append({"text": text, "label": 2}) # 2: Positif
        for text in neg_templates: train_data.append({"text": text, "label": 0}) # 0: Negatif
        for text in neu_templates: train_data.append({"text": text, "label": 1}) # 1: Netral

    # Membuat 400 Data Pengujian (Tanpa Label)
    test_sentences = [
        "Barang sampai dengan selamat", "Kecewa dengan pengiriman", "Standar saja sih",
        "Bagus banget!", "Kurang suka warnanya", "Oke lah untuk harga segini",
        "Biasa saja tidak ada yang spesial", "Layanan sangat ramah", "Produk rusak saat sampai"
    ]
    test_data = [test_sentences[i % len(test_sentences)] + f" (Sampel ke-{i})" for i in range(400)]

    return pd.DataFrame(train_data), pd.DataFrame(test_data, columns=["text"])

print("Menyiapkan dataset...")
train_df, test_df = generate_mock_data()

# ==========================================
# 2. KONFIGURASI MODEL & TOKENIZER
# ==========================================

# Menggunakan model mBERT (Multilingual) yang mendukung Bahasa Indonesia
MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# ==========================================
# 3. PROSES PELATIHAN (TRAINING)
# ==========================================

print(f"Menggunakan perangkat: {device}")
print("Memulai pelatihan model Transformer...")
train_dataset = SentimentDataset(train_df.text.values, train_df.label.values, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

model.train()
for epoch in range(2): # Menggunakan 2 epoch agar lebih cepat untuk demonstrasi
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Loss rata-rata: {total_loss/len(train_loader):.4f}")

# ==========================================
# 4. PELABELAN DATA PENGUJIAN (PREDIKSI)
# ==========================================

print("\nMelakukan pelabelan otomatis pada 400 data pengujian...")
model.eval()
predictions = []
label_map = {0: "Negatif", 1: "Netral", 2: "Positif"}

with torch.no_grad():
    for text in tqdm(test_df['text'], desc="Prediksi"):
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=64).to(device)
        outputs = model(**inputs)
        pred_label = torch.argmax(outputs.logits, dim=1).item()
        predictions.append(label_map[pred_label])

# Masukkan hasil prediksi ke dataframe
test_df['label_prediksi'] = predictions

# ==========================================
# 5. EKSPOR HASIL KE CSV
# ==========================================

output_file = "hasil_pengujian_sentimen.csv"
test_df.to_csv(output_file, index=False)

print(f"\n[SUKSES] Data pengujian telah diberi label.")
print(f"File tersimpan sebagai: {output_file}")

# Menampilkan 5 data pertama hasil prediksi
print("\nContoh Hasil Prediksi:")
print(test_df.head())

# ==========================================
# 6. OPSIONAL: DOWNLOAD OTOMATIS (KHUSUS GOOGLE COLAB)
# ==========================================
try:
    from google.colab import files
    print("\nMendownload file CSV otomatis...")
    files.download(output_file)
except ImportError:
    print("\n[INFO] Jalankan di Google Colab untuk fitur download otomatis.")
    print(f"Jika di komputer lokal, file berada di folder: {os.getcwd()}")

Menyiapkan dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Loss rata-rata: 0.0536

Melakukan pelabelan otomatis pada 400 data pengujian...


Prediksi: 100%|██████████| 400/400 [00:50<00:00,  7.94it/s]


[SUKSES] Data pengujian telah diberi label.
File tersimpan sebagai: hasil_pengujian_sentimen.csv

Contoh Hasil Prediksi:
                                         text label_prediksi
0  Barang sampai dengan selamat (Sampel ke-0)        Negatif
1      Kecewa dengan pengiriman (Sampel ke-1)        Negatif
2              Standar saja sih (Sampel ke-2)        Positif
3                 Bagus banget! (Sampel ke-3)        Negatif
4          Kurang suka warnanya (Sampel ke-4)        Negatif

Mendownload file CSV otomatis...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>